The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/pipeline-edge-publish/edge-classification-finserv).

## Classification Financial Services Edge Deployment Demonstration

This notebook will walk through building Wallaroo pipeline with a a Classification model deployed to detect the likelihood of credit card fraud, then publishing that pipeline to an Open Container Initiative (OCI) Registry where it can be deployed in other Docker and Kubernetes environments.

This demonstration will focus on deployment to the edge.  For further examples of using Wallaroo with this computer vision models, see [Wallaroo 101](https://docs.wallaroo.ai/wallaroo-101/).

This demonstration will perform the following:

1. As a Data Scientist:
    1. Upload a computer vision model to Wallaroo, deploy it in a Wallaroo pipeline, then perform a sample inference.
    1. Publish the pipeline to an Open Container Initiative (OCI) Registry service.  This is configured in the Wallaroo instance.  See [Edge Deployment Registry Guide](https://staging.docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.  This demonstration uses a GitHub repository - see [Introduction to GitHub Packages](https://docs.github.com/en/packages/learn-github-packages/introduction-to-github-packages) for setting up your own package repository using GitHub, which can then be used with this tutorial.
    1. View the pipeline publish details.
1. As a DevOps Engineer:
    1. Deploy the published pipeline into an edge instance.  This example will use Docker.
    1. Perform a sample inference into the deployed pipeline with the same data used in the data scientist example.

## Data Scientist Pipeline Publish Steps

### Load Libraries

The first step is to import the libraries used in this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import pandas as pd
import requests


# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [7]:
import string
import random

# make a random 4 character suffix to verify uniqueness in tutorials
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

workspace_name = f'edge-publish-demo{suffix}'
pipeline_name = 'edge-pipeline'
xgboost_model_name = 'ccfraud-xgboost'
xgboost_model_file_name = './models/xgboost_ccfraud.onnx'
keras_model_name = 'ccfraud-keras'
keras_model_file_name = './models/keras_ccfraud.onnx'

In [8]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [9]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'edge-publish-demojohn', 'id': 7, 'archived': False, 'created_by': 'aa707604-ec80-495a-a9a1-87774c8086d5', 'created_at': '2023-09-08T17:24:44.472865+00:00', 'models': [{'name': 'ccfraud', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 8, 17, 24, 44, 752280, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 8, 17, 24, 44, 752280, tzinfo=tzutc())}], 'pipelines': [{'name': 'edge-pipeline', 'create_time': datetime.datetime(2023, 9, 8, 17, 24, 44, 975233, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in ONNX format, which is specified in the `framework` parameter.

In [10]:
xgboost_edge_demo_model = wl.upload_model(
    xgboost_model_name,
    xgboost_model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
)

keras_edge_demo_model = wl.upload_model(
    keras_model_name,
    keras_model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
)

### Reserve Pipeline Resources

Before deploying an inference engine we need to tell wallaroo what resources it will need.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be.

We will be testing this deployment for an edge scenario, so the resource specifications are kept small -- what's the minimum needed to meet the expected load on the planned hardware.

- cpus - 0.5 => allow the engine to use 4 CPU cores when running the neural net
- memory - 900Mi => each inference engine will have 2 GB of memory, which is plenty for processing a single image at a time.
- arch - we will specify the X86 architecture.

In [11]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("900Mi").build()

### Simulated Edge Deployment

We will now deploy our pipeline into the current Kubernetes environment using the specified resource constraints. This is a "simulated edge" deploy in that we try to mimic the edge hardware as closely as possible.

In [12]:
pipeline = get_pipeline(pipeline_name)
display(pipeline)

# clear the pipeline if previously run
pipeline.clear()
pipeline.add_model_step(xgboost_edge_demo_model)

pipeline.deploy(deployment_config = deploy_config)

name,edge-pipeline
created,2023-09-08 17:30:28.893020+00:00
last_updated,2023-09-08 18:21:00.097662+00:00
deployed,False
tags,
versions,"2d8f9e1d-dc65-4e90-a5ce-ee619162d8cd, 1ea2d089-1127-464d-a980-e087d1f052e2"
steps,ccfraud
published,True


Waiting for deployment - this will take up to 45s ............. ok


name,edge-pipeline
created,2023-09-08 17:24:44.975233+00:00
last_updated,2023-09-08 19:48:24.930788+00:00
deployed,True
tags,
versions,"02679ba1-a2d0-47bb-83cb-4ccdcd340b23, 873582f4-4b39-4a69-a2b9-536a0e29927c, 079cf5a1-7e95-4cb7-ae40-381b538371db"
steps,ccfraud
published,True


In [14]:
xgboost_pipeline_version = pipeline.versions()[0]
xgboost_pipeline_version

name,edge-pipeline
version,02679ba1-a2d0-47bb-83cb-4ccdcd340b23
creation_time,2023-08-Sep 19:48:24
last_updated_time,2023-08-Sep 19:48:24
deployed,False
tags,
steps,ccfraud-xgboost


### Run Single Image Inference

A single image, encoded using the Apache Arrow format, is sent to the deployed pipeline. Arrow is used here because, as a binary protocol, there is far lower network and compute overhead than using JSON. The Wallaroo Server engine accepts both JSON, pandas DataFrame, and Apache Arrow formats.

The sample DataFrames and arrow tables are in the `./data` directory.  We'll use the Apache Arrow table `cc_data_10k.arrow`.

Once complete, we'll display how long the inference request took.

In [15]:
with pa.ipc.open_file("./data/cc_data_10k.arrow") as f:
    data = f.read_all()

results = pipeline.infer(data, dataset=["*", "metadata.elapsed"])
elapsed = results['metadata.elapsed'][0].as_py()[1] / 1000000.0

print(f"Elapsed: {elapsed} ms")

Elapsed: 8.604759 ms


### Redeploy with Keras version

We will now redeploy the pipeline with the converted keras version of our model, then set that version of the pipeline.

In [19]:
# clear the pipeline if previously run
pipeline.clear()
pipeline.add_model_step(keras_edge_demo_model)
pipeline.deploy(deployment_config = deploy_config)

# give a moment to verify the model swap
import time
time.sleep(10)

keras_pipeline_version = pipeline.versions()[0]
keras_pipeline_version

 ok


name,edge-pipeline
version,55bb733e-cf49-4dea-87b0-79d561c3891e
creation_time,2023-08-Sep 19:49:41
last_updated_time,2023-08-Sep 19:49:41
deployed,False
tags,
steps,ccfraud-keras


In [20]:
# rerun the inferences with the new model

with pa.ipc.open_file("./data/cc_data_10k.arrow") as f:
    data = f.read_all()

results = pipeline.infer(data, dataset=["*", "metadata.elapsed"])
elapsed = results['metadata.elapsed'][0].as_py()[1] / 1000000.0

print(f"Elapsed: {elapsed} ms")

Elapsed: 1.788031 ms


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://staging.docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)` which has the following parameters and returns.

#### Publish a Pipeline Parameters

The `publish` method takes the following parameters.  The containerized pipeline will be pushed to the Edge registry service with the model, pipeline configurations, and other artifacts needed to deploy the pipeline.

| Parameter | Type | Description |
|---|---|---|
| `deployment_config` | `wallaroo.deployment_config.DeploymentConfig` (*Optional*) | Sets the pipeline deployment configuration.  For example:    For more information on pipeline deployment configuration, see the [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration]({{<ref "wallaroo-sdk-essentials-pipeline-deployment-config">}}).

#### Publish a Pipeline Returns

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline version id | integer | Numerical Wallaroo id of the pipeline version published. |
| status | string | The status of the pipeline publication.  Values include:  <ul><li>PendingPublish: The pipeline publication is about to be uploaded or is in the process of being uploaded.</li><li>Published:  The pipeline is published and ready for use.</li></ul> |
| Engine URL | string | The URL of the published pipeline engine in the edge registry. |
| Pipeline URL | string | The URL of the published pipeline in the edge registry. |
| Helm Chart URL | string | The URL of the helm chart for the published pipeline in the edge registry. |
| Helm Chart Reference | string | The help chart reference. |
| Helm Chart Version | string | The version of the Helm Chart of the published pipeline.  This is also used as the Docker tag. |
| Engine Config | `wallaroo.deployment_config.DeploymentConfig` | The pipeline configuration included with the published pipeline. |
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [21]:
xgb_pub=xgboost_pipeline_version.publish(deploy_config)
display(xgb_pub)
keras_pub=keras_pipeline_version.publish(deploy_config)
display(keras_pub)

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing...Published.


ID,18
Pipeline Version,02679ba1-a2d0-47bb-83cb-4ccdcd340b23
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:02679ba1-a2d0-47bb-83cb-4ccdcd340b23
Helm Chart URL,ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:02ef587731267bd36fc8f6b244eae9df52fbb43f9ecd79aee1082479cecb6cf8
Helm Chart Version,0.0.1-02679ba1-a2d0-47bb-83cb-4ccdcd340b23
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}}}, 'engineAux': {'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}}}}"
User Images,[]
Created By,john.hummel@wallaroo.ai


Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing...Published.


ID,19
Pipeline Version,55bb733e-cf49-4dea-87b0-79d561c3891e
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:55bb733e-cf49-4dea-87b0-79d561c3891e
Helm Chart URL,ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:9536e3869a9574c1f839e100f23deba018cf4dd048b1ca8466d376263daf0195
Helm Chart Version,0.0.1-55bb733e-cf49-4dea-87b0-79d561c3891e
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}}}, 'engineAux': {'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}}}}"
User Images,[]
Created By,john.hummel@wallaroo.ai


In [22]:
display(pipeline)

name,edge-pipeline
created,2023-09-08 17:24:44.975233+00:00
last_updated,2023-09-08 19:49:41.298059+00:00
deployed,True
tags,
versions,"55bb733e-cf49-4dea-87b0-79d561c3891e, 3d217006-de8f-435d-a50e-f498cc489ad8, 66ed071a-1b3c-4a90-b9a2-91891387c81b, 02679ba1-a2d0-47bb-83cb-4ccdcd340b23, 873582f4-4b39-4a69-a2b9-536a0e29927c, 079cf5a1-7e95-4cb7-ae40-381b538371db"
steps,ccfraud
published,True


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [23]:
wl.list_pipelines()

name,created,last_updated,deployed,tags,versions,steps,published
edge-cv-retail,2023-08-Sep 19:09:06,2023-08-Sep 19:19:11,False,,"28986e69-0fb2-429d-8a36-08c0609d40cf, be34623f-b651-4331-b0d6-bebb058437ae, d575f835-4f71-4fd3-a7c0-d01cfd624368, c4bb5321-7578-491b-a14b-42209450aee8, 77367cd0-16ea-459b-b27c-407c7f05b542, e31fdf85-e2ac-4264-8175-22e61efceb40",mobilenet,True
edgebiolabspipeline,2023-08-Sep 18:50:52,2023-08-Sep 19:02:06,True,,"b411c8aa-8368-484f-a615-d2a2bce634ca, 0321956b-098c-47eb-8f4c-3bd90e443f2d, 88bcc7a5-d618-4c72-90e5-651f1e252db9, d2425979-98ac-468c-83db-dd4f542e7217, 59a163a3-e9c7-4213-88bf-d732bcae7dbd, d608aa0f-4961-496f-b57c-ce02299b4e39, bf426f14-e3c4-4450-81d4-e833026505a9, b56f68b5-ae1d-49dc-b640-c964b10b117f, 951e4096-b3d7-426d-9eee-2b763d4a0558, e36b6b52-5652-440e-b856-89e42782b62f, 46ad2f62-987f-459d-8283-f495300647fa, 9c283c94-0ecd-4160-998e-b462d03008e1, 78b125a9-5311-46e7-adc8-5794f9ca29f0",edgebiolabsmodel,True
edge-cv-demo,2023-08-Sep 18:25:24,2023-08-Sep 18:37:14,False,,"69a912fb-47da-4049-98d5-aa024e7d66b2, 482fc033-00a6-42e7-b359-90611b76f74d, 32805f9a-40eb-4366-b444-635ab466ef76, b412ff15-c87b-46ea-8d96-48868b7867f0, aaf2c947-af26-4b0e-9819-f8aca5657017, 7ad0a22c-6472-4390-8f33-a8b3eccc7877, c73bbf20-8fe3-4714-be5e-e35773fe4779, fc431a83-22dc-43db-8610-cde3095af584",resnet-50,True
edge-pipeline,2023-08-Sep 17:30:28,2023-08-Sep 18:21:00,False,,"2d8f9e1d-dc65-4e90-a5ce-ee619162d8cd, 1ea2d089-1127-464d-a980-e087d1f052e2",ccfraud,True
edge-pipeline,2023-08-Sep 17:24:44,2023-08-Sep 19:49:41,True,,"55bb733e-cf49-4dea-87b0-79d561c3891e, 3d217006-de8f-435d-a50e-f498cc489ad8, 66ed071a-1b3c-4a90-b9a2-91891387c81b, 02679ba1-a2d0-47bb-83cb-4ccdcd340b23, 873582f4-4b39-4a69-a2b9-536a0e29927c, 079cf5a1-7e95-4cb7-ae40-381b538371db",ccfraud,True
edge-pipeline-classification-cybersecurity,2023-08-Sep 15:40:28,2023-08-Sep 15:45:22,False,,"60222730-4fb5-4179-b8bf-fa53762fecd1, 86040216-0bbb-4715-b08f-da461857c515, 34204277-bdbd-4ae2-9ce9-86dabe4be5f5, 729ccaa2-41b5-4c8f-89f4-fe1e98f2b303, 216bb86b-f6e8-498f-b8a5-020347355715",aloha,True
edge-pipeline,2023-08-Sep 15:36:03,2023-08-Sep 15:36:03,False,,"83b49e9e-f43d-4459-bb2a-7fa144352307, 73a5d31f-75f5-42c4-9a9d-3ee524113b6c",aloha,False
vgg16-clustering-pipeline,2023-08-Sep 14:52:44,2023-08-Sep 14:56:09,False,,"50d6586a-0661-4f26-802d-c71da2ceea2e, d94e44b3-7ff6-4138-8b76-be1795cb6690, 8d2a8143-2255-408a-bd09-e3008a5bde0b",vgg16-clustering,True


### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [24]:
pipeline.publishes()

id,pipeline_version_name,engine_url,pipeline_url,created_by,created_at,updated_at
3,873582f4-4b39-4a69-a2b9-536a0e29927c,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:873582f4-4b39-4a69-a2b9-536a0e29927c,john.hummel@wallaroo.ai,2023-08-Sep 17:24:59,2023-08-Sep 17:24:59
18,02679ba1-a2d0-47bb-83cb-4ccdcd340b23,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:02679ba1-a2d0-47bb-83cb-4ccdcd340b23,john.hummel@wallaroo.ai,2023-08-Sep 19:49:51,2023-08-Sep 19:49:51
19,55bb733e-cf49-4dea-87b0-79d561c3891e,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:55bb733e-cf49-4dea-87b0-79d561c3891e,john.hummel@wallaroo.ai,2023-08-Sep 19:50:07,2023-08-Sep 19:50:07


## DevOps - Pipeline Edge Deployment

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.

### Docker Deployment

First, the DevOps engineer must authenticate to the same OCI Registry service used for the Wallaroo Edge Deployment registry.

For more details, check with the documentation on your artifact service.  The following are provided for the three major cloud services:

* [Set up authentication for Docker](https://cloud.google.com/artifact-registry/docs/docker/authentication)
* [Authenticate with an Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli)
* [Authenticating Amazon ECR Repositories for Docker CLI with Credential Helper](https://aws.amazon.com/blogs/compute/authenticating-amazon-ecr-repositories-for-docker-cli-with-credential-helper/)

For the deployment, the engine URL is specified with the following environmental variables:

* `DEBUG` (true|false): Whether to include debug output.
* `OCI_REGISTRY`: The URL of the registry service.
* `CONFIG_CPUS`: The number of CPUs to use.
* `OCI_USERNAME`: The edge registry username.
* `OCI_PASSWORD`:  The edge registry password or token.
* `PIPELINE_URL`: The published pipeline URL.

#### Docker Deployment Example

Using our sample environment, here's sample deployment using Docker with a computer vision ML model, the same used in the [Wallaroo Use Case Tutorials Computer Vision: Retail]({{<ref "use-case-computer-vision-retail">}}) tutorials.

```bash
docker run -p 8080:8080 \
    -e DEBUG=true -e OCI_REGISTRY={your registry server} \
    -e CONFIG_CPUS=4 \
    -e OCI_USERNAME=oauth2accesstoken \
    -e OCI_PASSWORD={registry token here} \
    -e PIPELINE_URL={your registry server}/pipelines/edge-cv-retail:bf70eaf7-8c11-4b46-b751-916a43b1a555 \
    {your registry server}/engine:v2023.3.0-main-3707
```

### Docker Compose Deployment

For users who prefer to use `docker compose`, the following sample `compose.yaml` file is used to launch the Wallaroo Edge pipeline.  This is the same used in the [Wallaroo Use Case Tutorials Computer Vision: Retail]({{<ref "use-case-computer-vision-retail">}}) tutorials.

```yml
services:
  engine:
    image: {Your Engine URL}
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: {Your Pipeline URL}
      OCI_REGISTRY: {Your Edge Registry URL}
      OCI_USERNAME:  {Your Registry Username}
      OCI_PASSWORD: {Your Token or Password}
      CONFIG_CPUS: 4
```

For example:

```yml
services:
  engine:
    image: sample-registry.com/engine:v2023.3.0-main-3707
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: sample-registry.com/pipelines/edge-cv-retail:bf70eaf7-8c11-4b46-b751-916a43b1a555
      OCI_REGISTRY: sample-registry.com
      OCI_USERNAME:  _json_key_base64
      OCI_PASSWORD: abc123
      CONFIG_CPUS: 4
```

#### Docker Compose Deployment Example

The deployment and undeployment is then just a simple `docker compose up` and `docker compose down`.  The following shows an example of deploying the Wallaroo edge pipeline using `docker compose`.

```bash
docker compose up
[+] Running 1/1
 ✔ Container cv_data-engine-1  Recreated                                                                                                                                                                 0.5s
Attaching to cv_data-engine-1
cv_data-engine-1  | Wallaroo Engine - Standalone mode
cv_data-engine-1  | Login Succeeded
cv_data-engine-1  | Fetching manifest and config for pipeline: sample-registry.com/pipelines/edge-cv-retail:bf70eaf7-8c11-4b46-b751-916a43b1a555
cv_data-engine-1  | Fetching model layers
cv_data-engine-1  | digest: sha256:c6c8869645962e7711132a7e17aced2ac0f60dcdc2c7faa79b2de73847a87984
cv_data-engine-1  |   filename: c6c8869645962e7711132a7e17aced2ac0f60dcdc2c7faa79b2de73847a87984
cv_data-engine-1  |   name: resnet-50
cv_data-engine-1  |   type: model
cv_data-engine-1  |   runtime: onnx
cv_data-engine-1  |   version: 693e19b5-0dc7-4afb-9922-e3f7feefe66d
cv_data-engine-1  |
cv_data-engine-1  | Fetched
cv_data-engine-1  | Starting engine
cv_data-engine-1  | Looking for preexisting `yaml` files in //modelconfigs
cv_data-engine-1  | Looking for preexisting `yaml` files in //pipelines
```

### Helm Deployment

Published pipelines can be deployed through the use of helm charts.

Helm deployments take up to two steps - the first step is in retrieving the required `values.yaml` and making updates to override.

1. Pull the helm charts from the published pipeline.  The two fields are the Helm Chart URL and the Helm Chart version to specify the OCI .    This typically takes the format of:

  ```bash
  helm pull oci://{published.helm_chart_url} --version {published.helm_chart_version}
  ```

1. Extract the `tgz` file and copy the `values.yaml` and copy the values used to edit engine allocations, etc.  The following are **required** for the deployment to run:

  ```yml
  ociRegistry:
    registry: {your registry service}
    username:  {registry username here}
    password: {registry token here}
  ```

  Store this into another file, suc as `local-values.yaml`.

1. Create the namespace to deploy the pipeline to.  For example, the namespace `wallaroo-edge-pipeline` would be:

  ```bash
  kubectl create -n wallaroo-edge-pipeline
  ```

1. Deploy the `helm` installation with `helm install` through one of the following options:
    1. Specify the `tgz` file that was downloaded and the local values file.  For example:

        ```bash
        helm install --namespace {namespace} --values {local values file} {helm install name} {tgz path}
        ```

    1. Specify the expended directory from the downloaded `tgz` file.

        ```bash
        helm install --namespace {namespace} --values {local values file} {helm install name} {helm directory path}
        ```

    1. Specify the Helm Pipeline Helm Chart and the Pipeline Helm Version.

        ```bash
        helm install --namespace {namespace} --values {local values file} {helm install name} oci://{published.helm_chart_url} --version {published.helm_chart_version}
        ```

1. Once deployed, the DevOps engineer will have to forward the appropriate ports to the `svc/engine-svc` service in the specific pipeline.  For example, using `kubectl port-forward` to the namespace `ccfraud` that would be:

    ```bash
    kubectl port-forward svc/engine-svc -n ccfraud01 8080 --address 0.0.0.0`
    ```

The following code segment generates a `docker compose` template based on the previously published pipeline.

In [25]:
xgboost_docker_compose = f'''
services:
  engine:
    image: {xgb_pub.engine_url}
    ports:
      - 8081:8080
    environment:
      PIPELINE_URL: {xgb_pub.pipeline_url}
      OCI_USERNAME: YOUR USERNAME 
      OCI_PASSWORD: YOUR PASSWORD OR TOKEN
      OCI_REGISTRY: ghcr.io
      CONFIG_CPUS: 4
'''

print(xgboost_docker_compose)

keras_docker_compose = f'''
services:
  engine:
    image: {keras_pub.engine_url}
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: {keras_pub.pipeline_url}
      OCI_USERNAME: YOUR USERNAME 
      OCI_PASSWORD: YOUR PASSWORD OR TOKEN
      OCI_REGISTRY: ghcr.io
      CONFIG_CPUS: 4
'''

print(keras_docker_compose)


services:
  engine:
    image: ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:02679ba1-a2d0-47bb-83cb-4ccdcd340b23
      OCI_USERNAME: YOUR USERNAME 
      OCI_PASSWORD: YOUR PASSWORD OR TOKEN
      OCI_REGISTRY: ghcr.io
      CONFIG_CPUS: 4


services:
  engine:
    image: ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/standalone-mini:v2023.3.0-3798
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:55bb733e-cf49-4dea-87b0-79d561c3891e
      OCI_USERNAME: YOUR USERNAME 
      OCI_PASSWORD: YOUR PASSWORD OR TOKEN
      OCI_REGISTRY: ghcr.io
      CONFIG_CPUS: 4



## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `testboy.local`.  Replace this URL with the URL of you edge deployment.


In [8]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-pipeline","status":"Running"}]}

In [9]:
!curl testboy.local:8081/pipelines

{"pipelines":[{"id":"edge-pipeline","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

In [10]:
!curl testboy.local:8080/models

{"models":[{"name":"ccfraud-keras","sha":"bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507","status":"Running","version":"d07ed614-fb75-4052-93b5-2062e9efaa5e"}]}

In [11]:
!curl testboy.local:8081/models

{"models":[{"name":"ccfraud-xgboost","sha":"054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52","status":"Running","version":"8a7c525c-f509-43e2-8dbf-99b47d2f1f42"}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/pipelines/{pipeline-name}`:  The `pipeline-name` is the same as returned from the [`/pipelines`](#list-pipelines) endpoint as `id`.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing]({{<ref "wallaroo-sdk-essentials-pipeline#anomaly-testing">}}).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.


In [12]:
!curl -X POST testboy.local:8080/pipelines/edge-pipeline -H "Content-Type: application/vnd.apache.arrow.file" --data-binary @./data/cc_data_10k.arrow

[{"check_failures":[],"elapsed":[1839150,3424733],"model_name":"ccfraud-keras","model_version":"d07ed614-fb75-4052-93b5-2062e9efaa5e","original_data":null,"outputs":[{"Float":{"data":[0.993003249168396,0.993003249168396,0.993003249168396,0.993003249168396,0.001091688871383667,0.00047266483306884766,0.0008217096328735352,0.0011294782161712646,0.0018743574619293213,0.0011520087718963623,0.0016568303108215332,0.0010267198085784912,0.00019043684005737305,0.0003236532211303711,0.0006218552589416504,0.00033929944038391113,0.0009412467479705811,0.0004061460494995117,0.00014156103134155273,0.0006403028964996338,0.0008019506931304932,0.0011220276355743408,0.0007892251014709473,0.00040519237518310547,0.0008138120174407959,0.00046765804290771484,0.00020697712898254395,0.0007166266441345215,0.0007742643356323242,0.0007204711437225342,0.0011078417301177979,0.00034046173095703125,0.0009812712669372559,0.00014293193817138672,0.00011792778968811035,0.0006335675716400146,0.001365959644317627,0.00021669

In [13]:
!curl -X POST testboy.local:8081/pipelines/edge-pipeline -H "Content-Type: application/vnd.apache.arrow.file" --data-binary @./data/cc_data_10k.arrow

[{"check_failures":[],"elapsed":[1871869,5726242],"model_name":"ccfraud-xgboost","model_version":"8a7c525c-f509-43e2-8dbf-99b47d2f1f42","original_data":null,"outputs":[{"Float":{"data":[1.0094895362854004,1.0094895362854004,1.0094895362854004,1.0094895362854004,-1.8477439880371094e-6,-0.00004494190216064453,-0.00009369850158691406,-0.0000832676887512207,-0.00008344650268554688,0.0004897117614746094,0.0006609857082366943,0.00007578730583190918,-0.00010061264038085938,-0.000519871711730957,-3.7550926208496094e-6,-0.00010895729064941406,-0.00017696619033813477,-0.000028371810913085938,0.00002181529998779297,-0.00008505582809448242,-0.000049233436584472656,0.00020524859428405762,0.00001677870750427246,-0.00008910894393920898,0.00008806586265563965,0.0002167224884033203,-0.00003045797348022461,-0.0001615285873413086,-0.00007712841033935547,-8.046627044677734e-6,-0.000048100948333740234,-0.00013583898544311523,-0.000014126300811767578,-0.00015842914581298828,-0.00004863739013671875,0.0000731

We'll do the same thing, only this time submit the data in the form of pandas record format via the Requests library.

In [16]:
import json
import requests
import pandas as pd

# set the content type and accept headers
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

# Submit arrow file
dataFile="./data/cc_data_10k.df.json"

data = json.load(open(dataFile))

host = 'http://testboy.local:8080'

deployurl = f'{host}/pipelines/edge-pipeline'

response = requests.post(
                    deployurl, 
                    headers=headers, 
                    json=data, 
                    verify=True
                )

# # display the first 5 results back
display(pd.DataFrame(response.json()))


,check_failures,elapsed,model_name,model_version,original_data,outputs,pipeline_name,shadow_data,time
0,[],"[89274160, 2904404]",ccfraud-keras,d07ed614-fb75-4052-93b5-2062e9efaa5e,None,"[{'Float': {'data': [0.993003249168396, 0.9930...",edge-pipeline,{},1694203063387


In [17]:
import json
import requests
import pandas as pd

# set the content type and accept headers
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

# Submit arrow file
dataFile="./data/cc_data_10k.df.json"

data = json.load(open(dataFile))

host = 'http://testboy.local:8081'

deployurl = f'{host}/pipelines/edge-pipeline'

response = requests.post(
                    deployurl, 
                    headers=headers, 
                    json=data, 
                    verify=True
                )

# # display the first 5 results back
display(pd.DataFrame(response.json()))


,check_failures,elapsed,model_name,model_version,original_data,outputs,pipeline_name,shadow_data,time
0,[],"[90665562, 3993088]",ccfraud-xgboost,8a7c525c-f509-43e2-8dbf-99b47d2f1f42,None,"[{'Float': {'data': [1.0094895362854004, 1.009...",edge-pipeline,{},1694203064811
